# Hyperparameter Search

### Imports

In [14]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets
from sklearn import tree
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split as tts
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

import warnings; warnings.simplefilter('ignore')
from time import time

### Loading digit dataset and splitting in train and test

In [2]:
X, y = datasets.load_digits(return_X_y = True)

X_train, X_test, y_train, y_true = tts(X,y,test_size = 0.2, random_state = 42)

### Getting parameters for Decision Tree Classifier

In [3]:
tree.DecisionTreeClassifier().get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'presort': 'deprecated',
 'random_state': None,
 'splitter': 'best'}

### Creating param grid for Grid Search CV and finding the best parameters

In [4]:
param_grid = [
              {'splitter': ['best', 'random'], 
               'max_depth': [1,2,3,4,5,6,7,8,9,10], 
               'criterion': ['gini', 'entropy']}
              ]
decisionTreeclf = tree.DecisionTreeClassifier()
search = GridSearchCV(estimator = decisionTreeclf, param_grid=param_grid)
search.fit(X_train, y_train)
print(search.best_params_)

{'criterion': 'entropy', 'max_depth': 10, 'splitter': 'best'}


### Classification Report for best parameters

In [5]:
y_pred = search.predict(X_test)
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.94      0.93        33
           1       0.78      0.89      0.83        28
           2       0.81      0.76      0.78        33
           3       0.88      0.85      0.87        34
           4       0.91      0.87      0.89        46
           5       0.92      0.96      0.94        47
           6       0.97      0.94      0.96        35
           7       0.85      0.85      0.85        34
           8       0.90      0.87      0.88        30
           9       0.90      0.90      0.90        40

    accuracy                           0.89       360
   macro avg       0.88      0.88      0.88       360
weighted avg       0.89      0.89      0.89       360



### Selecting 'precision_micro', 'precision_macro', 'recall_micro', 'recall_macro','f1_micro', 'f1_macro' scores to predict and calculate the classification matrix

In [9]:
scores = ['precision_micro', 'precision_macro', 'recall_micro', 'recall_macro','f1_micro', 'f1_macro']

### Using GridSearchCV

In [ ]:
start = time()
for score in scores:
  print(f"# Tuning hyper-parameters for {score}")
  search = GridSearchCV(estimator=decisionTreeclf, param_grid=param_grid, scoring=score)
  search.fit(X_train,y_train)
  print("Best parameters set found on development set:")
  print()
  print(search.best_params_)
  print("Classification Report")
  print()
  y_pred = search.predict(X_test)
  print(classification_report(y_true, y_pred))
  print()
end = time()
print(f"Time taken for Grid Search CV on all the scores: {end - start}")

### Using RandomizedSearchCV

In [ ]:
n_iter = 20
for score in scores:
  print(f"# Tuning hyper-parameters for {score}")
  search = RandomizedSearchCV(estimator=decisionTreeclf, param_distributions=param_grid, scoring=score, n_iter=n_iter)
  search.fit(X_train,y_train)
  print("Best parameters set found on development set:")
  print()
  print(search.best_params_)
  print("Classification Report")
  print()
  y_pred = search.predict(X_test)
  print(classification_report(y_true, y_pred))
  print()
print(f"Time taken for Randomized Search CV on all the scores: {end - start}")

### Calculating time taken for Randomized Search with 20 iterations

In [24]:
clf = tree.DecisionTreeClassifier()
n_iter_search = 20
start = time()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid,
                                   n_iter=n_iter_search)
random_search.fit(X, y)
end = time()
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((end - start), n_iter_search))

RandomizedSearchCV took 1.12 seconds for 20 candidates parameter settings.


### Calculating time taken for Randomized Search with 360 (maximum) iterations

In [27]:
n_iter_search = 360
start = time()
random_search = RandomizedSearchCV(clf, param_distributions=param_grid,
                                   n_iter=n_iter_search)
random_search.fit(X, y)
end = time()
print("RandomizedSearchCV took %.2f seconds for %d candidates"
      " parameter settings." % ((end - start), n_iter_search))

RandomizedSearchCV took 2.10 seconds for 360 candidates parameter settings.


### Calculating time taken for Grid Search

In [28]:
start = time()
grid_search = GridSearchCV(clf, param_grid=param_grid)
grid_search.fit(X, y)
end = time()
print("GridSearchCV took %.2f seconds for"
      " parameter settings." % (end - start))

GridSearchCV took 2.09 seconds for parameter settings.


### Observations

* As we can specify the number of iterations in RandomizedSearchCV, we can complete hyperparameter search in less time using less iterations
* GridSearchCV iterates all the possible combinations so it will take more time than RandomizedSearchCV
* When number of iterations is maximum, RandomizedSearchCV will take almost same time as GridSearchCV